#### Read the genome neighborhood analysis result

In [2]:
import pickle
import pandas as pd

# Read the test file into a dataframe
df = pd.read_csv("../data/Other_Files/pfam_neighbors_PF02458.txt", delimiter='\t')
df.to_csv("../output/pfam_neighbors_PF02458.csv")

# Save the query and neighbor IDs into two separate txt files for the SSN submission
# Query
Query_ID = []
for id in df['Query ID'].to_list():
    Query_ID.append(id)
with open('../output/PF01494_with_PF02458.txt', 'w') as fp:
    for id in Query_ID:
        fp.write("%s\n" % id) # write each item on a new line

# Neighbor
Neighbor_ID = []
for id in df['Neighbor ID'].to_list():
    Neighbor_ID.append(id)
with open('../output/PF02458_with_PF01494.txt', 'w') as fp:
    for id in Neighbor_ID:
        fp.write("%s\n" % id) # write each item on a new line

# Create a "PF01494 to PF02458" dictionary and a "PF02458 to PF01494" dictionary and save them
PF01494_to_PF02458_dict = {}
PF02458_to_PF01494_dict = {}
for i, j in zip(Query_ID, Neighbor_ID):
    PF01494_to_PF02458_dict[i] = j
    PF02458_to_PF01494_dict[j] = i

with open("../output/PF01494_to_PF02458_dict.pkl", 'wb') as file_handle:
     pickle.dump(PF01494_to_PF02458_dict, file_handle)

with open("../output/PF02458_to_PF01494_dict.pkl", 'wb') as file_handle:
     pickle.dump(PF02458_to_PF01494_dict, file_handle)


#### Analyze the clusterings of PF01494 and PF02458 by the AMI score

In [6]:
import pickle
import pandas as pd
from sklearn.metrics.cluster import adjusted_mutual_info_score

# Read the PF01494 clustering file
df2 = pd.read_csv("../data/PF01494_SSN_Score150_cluster.txt", delimiter='\t')
df2.to_csv("../output/PF01494_SSN_Score150_cluster.csv")
PF01494_cluster_id = df2['UniProt ID'].to_list()
PF01494_cluster = df2['Cluster Number'].to_list()
PF01494_id_to_cluster = {}
for i, j in zip(PF01494_cluster_id, PF01494_cluster):
    PF01494_id_to_cluster[i] = j
    
# Read the PF02458 clustering file
df3 = pd.read_csv("../data/PF02458_SSN_Score150_cluster.txt", delimiter='\t')
df3.to_csv("../output/PF02458_SSN_Score150_cluster.csv")
df3['UniProt ID'].to_list()
PF02458_cluster_id = df3['UniProt ID'].to_list()
PF02458_cluster = df3['Cluster Number'].to_list()
PF02458_id_to_cluster = {}
for n, k in zip(PF02458_cluster_id, PF02458_cluster):
    PF02458_id_to_cluster[n] = k

# Read the PF01494_and_PF02458 dictionary
with open("../output/PF01494_to_PF02458_dict.pkl", 'rb') as file_handle:
    PF01494_to_PF02458_dict = pickle.load(file_handle)

# Create a list for PF01494 to PF02458 pairs that are not singletons
PF01494_and_PF02458 = []
for id in PF01494_cluster_id:
    if PF01494_to_PF02458_dict[id] in PF02458_cluster_id:
        PF01494_and_PF02458.append(id)
print("Total pairs: ", len(PF01494_and_PF02458))

updated_PF01494_cluster = []
updated_PF02458_cluster = []
for PF01494_id in PF01494_and_PF02458:
    updated_PF01494_cluster.append(PF01494_id_to_cluster[PF01494_id])
    updated_PF02458_cluster.append(PF02458_id_to_cluster[PF01494_to_PF02458_dict[PF01494_id]])

# Calculate the Adjusted Mutual Information (AMI) score
AMI = adjusted_mutual_info_score(updated_PF01494_cluster, updated_PF02458_cluster)
print("AMI score: ", AMI)


Total pairs:  167
AMI score:  0.8483212948915777


#### Compare the active site residues between ancestral ATs and some extant ATs

In [17]:
import numpy as np
import pandas as pd
import re
from Bio import SeqIO
from Bio.PDB import PDBParser, NeighborSearch

AA_RESIDUES = [
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
    'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP',
    'TYR', 'VAL'
]

aa_dict = {
    'ALA': 'A',  # Alanine
    'ARG': 'R',  # Arginine
    'ASN': 'N',  # Asparagine
    'ASP': 'D',  # Aspartic acid
    'CYS': 'C',  # Cysteine
    'GLN': 'Q',  # Glutamine
    'GLU': 'E',  # Glutamic acid
    'GLY': 'G',  # Glycine
    'HIS': 'H',  # Histidine
    'ILE': 'I',  # Isoleucine
    'LEU': 'L',  # Leucine
    'LYS': 'K',  # Lysine
    'MET': 'M',  # Methionine
    'PHE': 'F',  # Phenylalanine
    'PRO': 'P',  # Proline
    'SER': 'S',  # Serine
    'THR': 'T',  # Threonine
    'TRP': 'W',  # Tryptophan
    'TYR': 'Y',  # Tyrosine
    'VAL': 'V'   # Valine
}

def is_amino_acid(residue):
    return residue.get_resname() in AA_RESIDUES

def get_centroid(residues):
    coords = [atom.get_coord() for residue in residues for atom in residue]
    active_site_center = np.mean(coords, axis=0)
    return active_site_center

def get_ligand_centroid(pdb_filename, ligand_resname):
    pattern = r'(?<=\s)-?\d{1,3}\.\d{3}(?=\s|$)'
    with open(pdb_filename, "r") as file:
        ligand_lines = [line.strip() for line in file if line.startswith("HETATM") and ligand_resname in line]
        coordinates = np.empty((0, 3))
        for line in ligand_lines:
            matched_strings = re.findall(pattern, line)
            x_y_z = [float(x) for x in matched_strings[:3]]
            coordinates = np.vstack((coordinates, x_y_z))
    active_site_center = np.mean(coordinates, axis=0)
    return active_site_center

def find_residues_by_distance(atoms, center, distance):
    neighbor_search = NeighborSearch(atoms)
    nearby_atoms = neighbor_search.search(center, distance)
    residues = set(atom.get_parent() for atom in nearby_atoms if is_amino_acid(atom.get_parent()))
    return list(residues)

def fasta_to_dict(fasta_file):
    '''Reads a FASTA file and returns a dictionary with sequence IDs as keys and sequences as values'''
    sequences = {}
    with open(fasta_file, 'r') as f:
        for record in SeqIO.parse(f, 'fasta'):
            sequences[record.id] = str(record.seq)
    return sequences

def build_mapping(query_sequence):
    '''Builds a dictionary mapping query positions to alignment positions'''
    mapping = {}
    count = 0
    for i, res in enumerate(query_sequence):
        if res != "-":
            count += 1
            mapping[count] = i
    return mapping

def get_corresponding_residues(sequences, mapping, query_positions):
    '''Given a list of query positions, retrieves the residues in other sequences'''
    residues = {}
    for seq_id, seq in sequences.items():
        residues[seq_id] = [seq[mapping[pos]] for pos in query_positions]
    return residues

# Alignment file:
fasta_filename = '../data/CazE_anc451_clade_seqs_aligned.fasta' #@param {type:"string"}

# Query seq:
query_id = 'Anc451' #@param {type:"string"}

# Structure file:
pdb_filename = '../data/Anc451.pdb'

# Select how to determine the center:
center = 'key_residues' # "ligand_substrate" or "key_residues"

# Select the chain of interest:
chain_of_interest = 'A'

# Ligand/substrate/cofactor name in the pdb file if "ligand_substrate" is selected:
ligand_resname = '' 

# The chain where the ligand/substrate/cofactor is located at (A, B... or blank if unspecified):
chain_of_ligand_substrate = 'A'

# Select a targeted number of residues:
num_residues = 96

# Set the max distance to be a big number if you do not want to restrict the search by the max distance.
max_distance = 30

# Parse the PDB file
parser = PDBParser()
structure = parser.get_structure("protein", pdb_filename)
model = structure[0]

if center == 'ligand_substrate':

    if chain_of_ligand_substrate == '':
        active_site_center = get_ligand_centroid(pdb_filename, ligand_resname)

    else:
        ligand = next((res for res in model[chain_of_ligand_substrate].get_residues() if res.get_resname() == ligand_resname), None)
        active_site_center = get_centroid([ligand]) if ligand else sys.exit("Error: Ligand not found.")

else:
    residue_numbers = input("Enter active site residue numbers (comma-separated): ").split(', ')
    residues = [model[chain_of_interest][int(num.strip())] for num in residue_numbers]
    active_site_center = get_centroid(residues)

atoms = list(model[chain_of_interest].get_atoms())

distance_step = 0.5
current_distance = 5.0
nearby_residues = find_residues_by_distance(atoms, active_site_center, current_distance)

while len(nearby_residues) < num_residues and current_distance < max_distance:
    current_distance += distance_step
    nearby_residues = find_residues_by_distance(atoms, active_site_center, current_distance)

if len(nearby_residues) >= num_residues:
    residue_distances = [(res, np.linalg.norm(list(res.get_atoms())[0].get_coord() - active_site_center)) for res in nearby_residues]

    # Sort the residue_distances based on the distance
    sorted_residue_distances = sorted(residue_distances, key=lambda x: x[1])

    # Extract the distances from the sorted_residue_distances
    sorted_distances = [dist for _, dist in sorted_residue_distances]

    # Get the required number of residues
    final_residues = [res for res, _ in sorted_residue_distances[:num_residues]]

    # Create a residue number to distance dictionary
    pos_to_distance_dict = {}
    for i, j in zip(final_residues, sorted_distances):
        pos_to_distance_dict[i.id[1]] = j

# Get the fasta file
sequences = fasta_to_dict(fasta_filename)

# Build the mapping dictionary
mapping = build_mapping(sequences[query_id])

# Get positions from the residues we identified from the protein structure
query_positions = [res.id[1] for res in final_residues]

# Get residues from other sequences in the alignment
residues_from_alignment = get_corresponding_residues(sequences, mapping, query_positions)

residue_data = []
for res in final_residues:
    res_data = {
        'Chain': res.parent.id,
        'Residue Name': aa_dict[res.resname],
        'Residue Number': int(res.id[1]),
        'Residue Distance': "{:.1f}".format(pos_to_distance_dict[res.id[1]])
    }
    for seq_id, res_list in residues_from_alignment.items():
        pos = query_positions.index(res.id[1])  # index of the residue in the query_positions list
        res_data[seq_id] = res_list[pos]  # This adds a column named seq_id with the corresponding residue as the value
    residue_data.append(res_data)

df = pd.DataFrame(residue_data)

# Customized output:
# Order: 'AncAT4', 'AncAT5', 'KAH6850668.1', 'KAH6641043.1', 'CazE',  'AncAT6',	RYP17742.1	TpAT	KAF7554211.1
df_cutomized = df[['Residue Number', 'Residue Distance', 'Anc451', 'Anc452', 'KAH6850668.1', 'KAH6641043.1', 'XP_001225293.1', 'Anc454', 'RYP17742.1', 'XP_046069174.1', 'KAF7554211.1']]

# Initialize a list to hold the sum of differences, with a length that matches the number of DataFrame columns
sum_of_differences = [None] * 11

# Function to count differences between two values
def count_differences(value1, value2):
    return sum(a != b for a, b in zip(value1, value2))

# Calculate the sum of differences and store them in the list
for i in range(3, 11):  # Assuming you want to compare columns 4-11 with column 3
    # Calculate the differences row-wise and sum them
    sum_of_differences[i] = df_cutomized.apply(lambda row: count_differences(row.iloc[2], row.iloc[i]), axis=1).sum()

# Append the sum of differences as a new row to the DataFrame
df_cutomized = df_cutomized.append(pd.Series(sum_of_differences, index=df_cutomized.columns), ignore_index=True)

df_cutomized.to_excel('../output/active_site_comparison.xlsx')
df_cutomized

/Users/chad/miniforge3/envs/ete3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 7035
  warnings.warn(
/var/folders/jz/g6qwk6rs71nfx1y5xqrr_0j40000gn/T/ipykernel_44757/1579104604.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cutomized = df_cutomized.append(pd.Series(sum_of_differences, index=df_cutomized.columns), ignore_index=True)


,Residue Number,Residue Distance,Anc451,Anc452,KAH6850668.1,KAH6641043.1,XP_001225293.1,Anc454,RYP17742.1,XP_046069174.1,KAF7554211.1
0,166.0,3.1,H,H,H,H,H,H,H,H,H
1,167.0,3.7,T,T,A,T,T,T,T,T,T
2,38.0,5.2,T,T,T,T,T,T,T,T,T
3,169.0,5.4,M,M,M,M,M,M,M,M,M
4,170.0,5.5,D,D,D,D,D,D,D,D,D
...,...,...,...,...,...,...,...,...,...,...,...
92,202.0,16.9,R,R,R,R,R,R,R,R,R
93,72.0,16.9,W,W,W,W,W,W,W,W,W
94,387.0,17.8,T,T,T,M,M,T,T,T,T
95,206.0,18.0,I,I,I,I,I,I,I,I,I
